In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


In [2]:
#Assignment1
flight = spark.read.option("inferschema","true").option("header","true").csv("flight.csv")

#datatypes
flight.printSchema()

#max flights
flight.sort(desc('count')).show(1)

#flights>10
flight.filter(col('count')>10).count()
#sql - SELECT count(*) FROM flighttable WHERE count>10

#flights with united states destination
#flight.filter(col("DEST_COUNTRY_NAME")=="United States").groupBy("DEST_COUNTRY_NAME").sum().show()
flight.select("DEST_COUNTRY_NAME","count").filter(col("DEST_COUNTRY_NAME")=="United States").groupBy("DEST_COUNTRY_NAME").sum().show()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
+-----------------+-------------------+------+
only showing top 1 row

+-----------------+----------+
|DEST_COUNTRY_NAME|sum(count)|
+-----------------+----------+
|    United States|    411352|
+-----------------+----------+



In [3]:
#map vs #flatmap

maap = sc.textFile("sample.txt")
map_result = maap.map(lambda line:line.split(" "))
map_result.collect()

fmap_result = maap.flatMap(lambda line:line.split(" "))
fmap_result.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/C:/Users/HI/sample.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:297)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:239)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:325)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.api.python.Py

In [ ]:
#assignment 2
youtube = spark.read.option("inferschema","true").option("header","true").csv("DEvideos.csv")

#select columns
videoStats = youtube.select("video_id","trending_date","title","views")

#create new column with views/100
videoStats.withColumn("new",col("views")/100)

#mean
videoStats.select(mean("views")).show()

#population mean
videoStats.select(stddev_pop("views")).show()

#maximum
videoStats.select(max("views")).show()

#group and aggregate by mean
videoStatGroup = videoStats.groupBy("trending_date").agg(mean("views"))

In [ ]:
#assignment 3
retail = spark.read.option("inferschema","true").option("header","true").csv("retail.csv")

 
#how many orders customer perform at which hour

#sql query
retail.createTempView("stocks") 
spark.sql("SELECT SUM(Quantity) AS Quantity,SUBSTRING(InvoiceDate,12,2) AS Hour FROM stocks GROUP BY Hour ORDER By Hour ASC")

#spark query
retail.select('Quantity',substring('InvoiceDate',12,2).alias("Hour")).groupBy("Hour").sum("Quantity").orderBy("Hour")

#How frequently was each product bought in different countries?

#sql query
spark.sql("SELECT Country,COUNT(StockCode) AS Products FROM stocks GROUP BY country")

#spark query
retail.groupBy("Country").agg(count("StockCode").alias("Products"))

#slice 
slice = retail.where(col("Country") == "United States")
#wrong
#us_items = datacube.slice(col("Country") == "United States")
#slice = datacube.select("Country").where("United States")
#us_items = datacube.get("Country" == "United States")

#dice
dice = retail.where(((col("Country") == "United States") | (col("Country") == "United Kingdom")) & (col("Quantity") == 0))
dicee = retail.where((col("Country") == "United States") | (col("Country") == "United Kingdom")).where(col("Quantity") == 0)
#wrong
#subcube = retail.dice(((col("Country") == "United States") | (col("Country") == "United Kingdom")) & (col("Quantity") == 0))
#subcube = retail.get(((col("Country") == "United States") | (col("Country") == "United Kingdom")) & (col("Quantity") == 0))

#RollUp
roll_up = retail.select("StockCode", "Country", (col("Quantity") > 0).alias("positive quantity"))
#create new column with all boolen vals as true
#wrong
#roll_upp = retail.rollup("Stock Code", "Country", (col("Quantity") > 0).alias("positive quantity"))
#roll_uppp = retail.where(col("Quantity") > 0)




In [ ]:
#Assignment4
#Pivot
# How many instances of each product were sold in each country?

retail.groupBy("StockCode").pivot("Country").sum("Quantity") #(shows total quantity(sum) in each country)

#wrong 
#retail.groupBy("StockCode").pivot("Country").count().show() (shows count of quantity in each country)
#retail.groupBy("Country").pivot("StockCode").sum("Quantity").show()
#retail.groupBy("InvoiceNo").pivot("Country").sum("Quantity").show()
#retail.groupBy("InvoiceNo").pivot("Country").count().show()

#joins 1
#Which employees could convince customers to order products?

In [ ]:
#assignment 6
flight.sort("count").explain()

#2008 airplane flight data
#https://www.kaggle.com/zernach/2018-airplane-flights
#flight.groupBy("Dest").sum("miles").withColumnRenamed("sum(miles)", "destination_total").sort("destination_total").limit(5).explain()

flight.where((col('DEST_COUNTRY_NAME').isNotNull()) & (col('DEST_COUNTRY_NAME')=='United States')).groupBy("ORIGIN_COUNTRY_NAME").sum("count").sort('ORIGIN_COUNTRY_NAME').explain()

In [ ]:
words = sc.parallelize (
["Big Data", "Data Science", "Intro to Web",  "Web Engineering",  "Network Theory", "Machine Learning", ]
)
words_map = words.filter(lambda x: "Data" in x)
words_map.collect()

In [ ]:
words = sc.parallelize (
["Big Data",
"Data Science",
"Intro to Web",
"Web Engineering",
"Network Theory",
"Machine Learning",
]
)
words_map = words.map(lambda x: (x))
words_map.collect()

In [4]:
spark.sql("SELECT * FROM employees e LEFT OUTER JOIN orders o ON e.employee_id = o.salesman_id").show()

AnalysisException: Table or view not found: employees; line 1 pos 14;
'Project [*]
+- 'Join LeftOuter, ('e.employee_id = 'o.salesman_id)
   :- 'SubqueryAlias e
   :  +- 'UnresolvedRelation [employees], [], false
   +- 'SubqueryAlias o
      +- 'UnresolvedRelation [orders], [], false
